# Construyendo y entrenando redes neuronales con PyTorch

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import numpy as np
import sklearn as skl
from torchviz import make_dot
import torch.optim as optim

### Ejemplo 1: modelo simple, con numpy

Nos interesa aprender un simple modelo de regresión lineal

$$ y = a + bx $$

Comenzamos por generar los datos.
Son $n=100$ puntos, en donde los *features* vienen representados por algunos valores $x_1,x_2,...$ de`x` y los *labels* por correspondientes valores $y_1,y_2,...$ de `y` dados por

$$ y_i = a + bx_i + \epsilon_i $$

donde $a=1$, $b=2$ y $\epsilon_i \sim \mathcal{N}(0,0.1)$ es un número aleatorio generado de una distribución gaussia de media 0 y varianza 0.1.

In [ ]:
# Generamos datos aleatorios
np.random.seed(42)
x = np.random.rand(100, 1) # features
y = 1 + 2 * x + .1 * np.random.randn(100, 1) # labels

Luego de aleatorizar el orden de los puntos, dividimos los datos en un conjunto de entrenamiento (de tamaño 80) y uno de validación (de tamaño 20).

In [ ]:
# Aleatorizamos el orden de las muestras
idx = np.arange(100)
np.random.shuffle(idx)

# Usamos las primeras 80 muestras para entrenamiento
train_idx = idx[:80]
# y las que 20 que quedan para validación
val_idx = idx[80:]

# Generamos los conjuntos (datasets) de entrenamientos y validación
x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val = x[val_idx], y[val_idx]

In [ ]:
# Visualicemos los datasets
fig,axes=plt.subplots(1,2)
fig.set_size_inches(5.0,3.0)
ax = axes[0]
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("training")
ax.scatter(x_train,y_train)
ax = axes[1]
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_title("validation")
ax.scatter(val_idx,val_idx,c='r')
fig.tight_layout()
plt.show()

El objetivo último es aprender los valores de los parámetros $a$ y $b$ usados para generar los datos.
Para ello consideramos la función de pérdida definida por el Error Cuadrático Medio entre las predicciones $\hat{y}_i:=y(x_i)$ y los valores labels $y_i$

\begin{eqnarray}
L(x,y;a,b) &=& \frac{1}{n}\sum_{i=1}^n (y_i-y(x_i))^2 \\
&=& \frac{1}{n}\sum_{i=1}^n (y_i-a-bx_i)^2 \\
\end{eqnarray}

Usaremos el método del descenso por el gradiente

$$ \frac{\partial L^{(t)}}{\partial a}(x,y;a,b) = -\frac{2}{n}\sum_{i=1}^n(y_i-y_i(x_i)) $$

$$ \frac{\partial L^{(t)}}{\partial b}(x,y;a,b) = -\frac{2}{n}\sum_{i=1}^nx_i(y_i-y_i(x_i)) $$

arrancando con valores iniciales de los parámetros $a=a^{(0)}$ y $b=b^{(0)}$ elegidos al azar de alguna distribución de probabilidades, para luego iterar sobre la época de entrenamiento $t$ según

$$ a^{(t+1)} = a^{(t)} - \eta \frac{\partial L^{(t)}}{\partial a}(x,y;a^{(t)},b^{(t)}) $$

$$ b^{(t+1)} = b^{(t)} - \eta \frac{\partial L^{(t)}}{\partial b}(x,y;a^{(t)},b^{(t)}) $$

donde el $\eta$ tal que $0<\eta\ll 1$, es la tasa de aprendizaje.

In [ ]:
# Inicializamos los parámetros "a" y "b" con valores elegidos al azar de una distribución normal
np.random.seed(42)
a = np.random.randn(1)
b = np.random.randn(1)

print(f"Iniciales a={a},b={b}")

# Seteamos la tasa de aprendizaje (learning rate eta)
lr = 1e-1

# Definimos el número de épocas de entrenamiento
n_epochs = 1000

# Iteramos sobre épocas de entrenamiento
for epoch in range(n_epochs):
    # Calculamos las predicciones del modelo sobre el conjunto de entrenamiento
    yhat = a + b * x_train
    
    # Calculamos el error de la predicción
    error = (y_train - yhat)
    # Calculamos la función de pérdida, en este caso, el error cuadrático medio (Mean Squared Error)
    loss = (error ** 2).mean()
    
    # Calculamos el gradiente de la función de pérdida con respecto a los parámetros; "a" y "b" en este caso
    a_grad = -2 * error.mean()
    b_grad = -2 * (x_train * error).mean()
    
    # Actualizamos los valores de los parámetros usando el gradiente y la tasa de aprendizaje
    a = a - lr * a_grad
    b = b - lr * b_grad

# Imprimimos los valores de "a" y "b" que resultan del ajuste
print(f"Ajustados a={a},b={b}")

# Corroboramos que todo anda bien, comparando con el ajuste proveido por scikit-learn
from sklearn.linear_model import LinearRegression
linr = LinearRegression()
linr.fit(x_train, y_train)
print(f"Check     a={linr.intercept_},b={linr.coef_[0]}")

### Ejemplo 2: idem, pero con PyTorch.
Convenientemente, PyTorch provee de `torch.autograd`, un motor de diferenciación automática para calcular gradientes.

Para ello, en este ejemplo tenemos que decirle a torch que `a` y `b` son tensores que requieren gradientes.

In [ ]:
# Nuestros datos estan en arrays de numpy.
# Los transformamos, entonces, a tensores de PyTorch.
x_train_tensor = torch.from_numpy(x_train).float()
y_train_tensor = torch.from_numpy(y_train).float()

# Inicializamos los parámetros "a" y "b" de manera similar a como lo hicimos con la versión numpy.
# Como aplicaremos el método del descenso por el gradiente sobre estos parámetros usando las funcionalidades
# proveídas por PyTorch, seteamos la opción "requires_grad" de estos tensores a "True".
a = torch.randn(1, requires_grad=True, dtype=torch.float)
b = torch.randn(1, requires_grad=True, dtype=torch.float)
print(f"Iniciales a={a.item()},b={b.item()}")
#print(f"Iniciales a={a},b={b}")

Entrenemos el modelo con un optimizador personalizado

In [ ]:
lr = 1e-1
n_epochs = 1000

torch.manual_seed(42)
a = torch.randn(1, requires_grad=True, dtype=torch.float) #, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float) #, device=device)
print(f"Iniciales a={a.item()},b={b.item()}")

# Iteramos sobre épocas de entrenamiento
for epoch in range(n_epochs):
    # Calculamos las predicciones del modelo, el error y la función de pérdida.
    yhat = a + b * x_train_tensor
    error = y_train_tensor - yhat
    loss = (error ** 2).mean()

    # Ya no necesitamos calcular gradientes de manera manual!
    # a_grad = -2 * error.mean()
    # b_grad = -2 * (x_tensor * error).mean()
    
    # Basta con decirle a PyTorch que llame al método backward() del tensor del cual queremos calcular el gradiente.
    # En este caso, dicho tensor es "loss"; la función de pérdida.
    loss.backward()
    # Veamos las componentes del gradiente que obtuvimos en la presente época
    print(f"epoch={epoch} a.grad={a.grad},b.grad={b.grad}")    
    
    # Para actualizar el valor de los parámetros, "a" y "b" en este caso, tenemos que desactivar el
    # cálculo automático de gradientes.
    # Esto es por la manera en que PyTorch está construido.
    with torch.no_grad():
        a -= lr * a.grad
        b -= lr * b.grad
    
    # PyTorch no resetea a cero los valores de las componentes del gradiente, sinó que procede de
    # manera acumulativa, ya que es conveniente por diferentes razones. 
    # Por ende, tenemos que decirle explícitamente que resetee los valores a cero.
    a.grad.zero_()
    b.grad.zero_()
    
# Imprimimos los valores ajustados
print(f"Ajustados a={a.item()},b={b.item()}")

Notar que hemos usado `with torch.no_grad()` al momento de actualizar los valores de los parámetros `a` y `b`.

Esto es así, porque de otra manera deberíamos cambiar el valor de `a.grad` y `b.grad` al actualizar los valores de `a` y `b`.
Por alguna razón, a PyTorch no le gusta esto (quizás para evitar evalauciones ciruclares) y, por ende, si querémos actualizar los valores de `a` y `b`, tenemos que decirle primero a PyTorch que desactive la actualización automática de los gradientes.

PyTorch calcula los gradientes de forma acumulativa.
Por ello, cada vez que usamos los gradientes para actualizar los parámetros, luego debemos resetear a 0 los gradientes usando, por ejemplo, el método `.zero()`.

En PyTorch los gradientes son actualizados tal como es especificado por el grafo computacional determinado por el modelo que uno define

![](comp-graph.png "")

In [ ]:
# A estos gráficos de arriba se los puede generar con torchviz
#make_dot(yhat)
#make_dot(error)
#make_dot(loss)

### Ejemplo 3: entrenando modelos con el optimizador proveido por PyTorch.

Además del cálculo automático de gradientes, PyTorch provee de un optimizador que se encarga de la actualización automática de parámetros.

In [ ]:
# Al igual que antes, inicializamos los parámetros "a" y "b" con valores aleatorios.
torch.manual_seed(42)
a = torch.randn(1, requires_grad=True, dtype=torch.float) #, device=device)
b = torch.randn(1, requires_grad=True, dtype=torch.float) #, device=device)
print(f"Iniciales a={a.item()},b={b.item()}")

# Definimos la tasa de aprendizaje y el número de épocas de entrenamiento
lr = 1e-1
n_epochs = 1000

# Instanciamos un optimizador tipo Stochastic Gradient Descent (SGD) para optimizar los valores de los parámetros.
# Notar que tenemos que especificarle cuales tensores son los parámetros sobre los cuales queremos optimizar.
optimizer = optim.SGD([a, b], lr=lr)

# Iteramos sobre épocas de entrenamiento
for epoch in range(n_epochs):
    # Calculamos predicciones, errores y la pérdida (error)
    yhat = a + b * x_train_tensor
    error = y_train_tensor - yhat
    loss = (error ** 2).mean()

    # Le pedimos a PyTorch que calcule el gradiente de la función "pérdida".
    loss.backward()    
    
    # Ya no necesitamos actualizar los valores de los parámetros a mano...
    # with torch.no_grad():
    #     a -= lr * a.grad
    #     b -= lr * b.grad
    # En cambio, invocamos al optimizador para que lo haga
    optimizer.step()
    
    # De la misma manera, ya no necesitamos resetear a cero los valores de las componentes del gradiente...
    # a.grad.zero_()
    # b.grad.zero_()
    # En cambio, invocamos al optimizador para que lo haga
    optimizer.zero_grad()
    
# Imprimimos los valores ajustados de los parámetros
print(f"Ajustados a={a.item()},b={b.item()}")